In [17]:
# =========================================================
# ASSET PRICING PROJECT 02: FAMA-FRENCH 3-FACTOR MODEL
# Author: Fares Awwad-Zeidan
# Level: Intermediate Asset Pricing
# =========================================================

# Ensure dependencies are present
# !pip install pandas-datareader

import pandas_datareader.data as web
import pandas as pd
import statsmodels.api as sm
import yfinance as yf

# ==========================================
# 1. DATA FETCHING (Stock)
# ==========================================
TICKER = 'AAPL' 
START_DATE = '2020-01-01'
END_DATE = '2024-01-01'

print(f"--- 1. Downloading Stock Data for {TICKER} ---")
# Download safely
raw_data = yf.download(TICKER, start=START_DATE, end=END_DATE, progress=False)

# Safe Column Selection
if 'Adj Close' in raw_data.columns:
    stock_data = raw_data['Adj Close']
else:
    stock_data = raw_data['Close']

# Resample to Monthly and --- CRITICAL FIX: REMOVE TIMEZONE ---
stock_returns = stock_data.resample('ME').last().pct_change().dropna()
stock_returns.index = stock_returns.index.tz_localize(None) # <--- THIS FIXES THE MERGE
stock_returns = pd.DataFrame(stock_returns)
stock_returns.columns = ['Portfolio_Return']

# ==========================================
# 2. DATA FETCHING (Fama-French)
# ==========================================
print(f"--- 2. Downloading Fama-French Factors ---")
try:
    ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=START_DATE, end=END_DATE)[0]
    ff_data = ff_data / 100 # Convert to decimal
    ff_data.index = ff_data.index.to_timestamp(freq='M') # Ensure format matches
except Exception as e:
    print("CRITICAL ERROR: Could not download Fama-French data.")
    print("Make sure pandas-datareader is installed: pip install pandas-datareader")
    raise e

# ==========================================
# 3. MERGE & REGRESS
# ==========================================
# Now the indices match (both are YYYY-MM-DD without timezones)
data = pd.merge(stock_returns, ff_data, left_index=True, right_index=True, how='inner')

if data.empty:
    print("ERROR: Data merge failed. The dates did not align.")
else:
    data['Excess_Return'] = data['Portfolio_Return'] - data['RF']

    # Regression
    X = data[['Mkt-RF', 'SMB', 'HML']]
    X = sm.add_constant(X)
    y = data['Excess_Return']

    model = sm.OLS(y, X).fit()

    # ==========================================
    # 4. REPORT
    # ==========================================
    print("\n" + "="*50)
    print(f"FAMA-FRENCH 3-FACTOR RESULTS: {TICKER}")
    print("="*50)
    print(model.summary())

    # Interpretation
    beta_hml = model.params['HML']
    print("\n" + "-"*50)
    print(f"VALUE FACTOR (HML): {beta_hml:.4f}")
    if beta_hml < 0:
        print("-> NEGATIVE. This is a GROWTH stock (High P/E, like Tech).")
    else:
        print("-> POSITIVE. This is a VALUE stock (Undervalued/Bank).")
    print("-" * 50)

--- 1. Downloading Stock Data for AAPL ---


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13856\3709751793.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_data = yf.download(TICKER, start=START_DATE, end=END_DATE, progress=False)


--- 2. Downloading Fama-French Factors ---


C:\Users\Administrator\AppData\Local\Temp\ipykernel_13856\3709751793.py:43: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=START_DATE, end=END_DATE)[0]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13856\3709751793.py:43: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=START_DATE, end=END_DATE)[0]



FAMA-FRENCH 3-FACTOR RESULTS: AAPL
                            OLS Regression Results                            
Dep. Variable:          Excess_Return   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.723
Method:                 Least Squares   F-statistic:                     41.00
Date:                Thu, 15 Jan 2026   Prob (F-statistic):           1.12e-12
Time:                        00:07:02   Log-Likelihood:                 78.428
No. Observations:                  47   AIC:                            -148.9
Df Residuals:                      43   BIC:                            -141.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0